# **NLP1_Run2**

In [1]:
!pip install openai
!pip install python-dotenv
!pip3 install pyomo
!apt install glpk-utils
!pip install glpk

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 325.5/325.5 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 17.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 6.1 MB/s eta 0:00:00
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libamd2 libcolamd2 libglpk40 libsuitesparseconfig5
Suggested packages:
  libiodbc2-dev
The following NEW packages will be installed:
  glpk-utils libamd2 libcolamd2 libglpk40 libsuitesparseconfig5
0 upgraded, 5 newly installed, 0 to remove and 45 not upgraded.
Need to get 625 kB of archives.
After this operation, 2,158 kB of additional disk space will be used.
Ge

In [2]:

import openai
import os
from IPython.display import Markdown


### **Accessing the GPT4 API**

In [3]:
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv('api_file.env'))
my_api_key = os.environ['api_key_env']
openai.api_key = my_api_key


### **Generate Mathematical Model**

In [4]:
problem = """A firm that packs refreshments and beers, situated in the province of Valencia (Spain) employs the same syrup to produce its 1.5 l COLI and PEPSA products on its S1 production line. Once processed, each hectolitre of syrup produces 40 units of the 1.5 l COLI product and 20 units of the 1.5 l PEPSA product. If X1 units of the 1.5 l COLI product and X2 units of the 1.5 l PEPSA product are produced, the firm estimates that the daily income obtained in dollars would be given by the following function:
49000 times X1 minus X1 squared plus 30 times X2 minus two times X2 squared.
It costs 150 dollars to buy and process each hectolitre of syrup. The S1 packaging line has a net capacity of producing 7100 1.5 l product units every hour. The firm works 5 days a week in 8h shifts. Given its weekly target coverage, the firm is committed to produce at least half the amount of PEPSA than COLI. Although priority orders tend to amend its production planning, the firm wishes to have a basic product planning that optimises its daily profits. """

In [5]:

client = openai.OpenAI(api_key=os.environ['api_key_env'])

response1 = client.chat.completions.create(
    messages=[
        {"role": "system", "content": "Please formulate only the variables for this mathematical optimization problem."},
        {
            "role": "user",
            "content": problem
    }
    ],
    model="gpt-4",
    seed = 3
)


In [6]:
print(response1.choices[0].message.content)

1. X1: Number of units of the 1.5 l COLI product produced
2. X2: Number of units of the 1.5 l PEPSA product produced.


In [7]:

response2 = client.chat.completions.create(
    messages=[
        {"role": "system", "content": "Please formulate only the objective function for this mathematical optimization problem."},
        {
            "role": "user",
            "content": problem + response1.choices[0].message.content
        }
    ],
    model="gpt-4",
    seed = 3
)

In [8]:
print(response2.choices[0].message.content)

The objective function to be maximized for daily profit can be specified as follows:

Profit(X1, X2) = 49000*X1 - X1^2 + 30*X2 - 2*X2^2 - 150*(X1/40 + X2/20)


In [9]:
response3 = client.chat.completions.create(
    messages=[
        {"role": "system", "content": "Please formulate only the constraints for this mathematical optimization problem."},
        {
            "role": "user",
            "content": problem + response1.choices[0].message.content +  response2.choices[0].message.content
        }
    ],
    model="gpt-4",
    seed = 3
)

In [10]:
print(response3.choices[0].message.content)

Constraints:

1) X1 ≥ 0, X2 ≥ 0 (Non-negativity constraint since we cannot produce a negative number of COLI and PEPSA units)

2) X2 ≥ 0.5*X1 (The company is committed to produce at least half the amount of PEPSA than COLI)

3) X1/40 + X2/20 ≤ (7100 * 8 * 5) (The company cannot exceed its weekly production capacity. Each hour it can produce 7100 units, so in an 8 hour shift for 5 days, the maximum total number of units it can produce is 7100 * 8 * 5)


### **Generate Pyomo Code**

In [11]:
response4 = client.chat.completions.create(
    messages=[
        {"role": "system", "content": "Please write a python pyomo code for this optimization problem. Use sample data where needed. Indicate where you use sample data."},
        {
            "role": "user",
            "content": response1.choices[0].message.content + response2.choices[0].message.content + response3.choices[0].message.content
        }
    ],
    model="gpt-4",
    seed = 3
)

In [12]:
Markdown(response4.choices[0].message.content)

Here is your optimization model in Pyomo.

```python
from pyomo.environ import *

# create a model
model = ConcreteModel()

# define decision variables
model.x1 = Var(within=NonNegativeReals)
model.x2 = Var(within=NonNegativeReals)

# define objective function
model.profit = Objective(expr = 49000*model.x1 - model.x1**2 + 30*model.x2 - 2*model.x2**2 - 150*((model.x1/40)+(model.x2/20)), sense = maximize)

# define constraints
model.Constraint1 = Constraint(expr = model.x2 >= 0.5*model.x1)
model.Constraint2 = Constraint(expr = model.x1/40 + model.x2/20 <= (7100 * 8 * 5))

# Solve
solver = SolverFactory('glpk')
solver.solve(model)

print("Optimal Solution:")
print(f"X1: {model.x1.value}")
print(f"X2: {model.x2.value}")
print(f"Profit: {model.profit()}")
```

Note:
1- Please install pyomo and glpk solver if they are not installed in your python environment. 
2- X1 and X2 can be fractional considering the constraints, if the need is for an integer result for these values then you'd need to use `model.x1 = Var(within=NonNegativeIntegers)` and `model.x2 = Var(within=NonNegativeIntegers)` respectively.

### **Run the code Generated by GPT4**

In [13]:
# Install Python API for AMPL
!pip install amplpy --upgrade

from amplpy import AMPL,ampl_notebook

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 28.4 MB/s eta 0:00:00


In [14]:
_ = load_dotenv(find_dotenv('second_knitro_license.env'))
my_knitro_license = os.environ['knitro_license']

ampl = ampl_notebook(
    modules=["knitro"],  # modules to install
    license_uuid=my_knitro_license,  # license to use
)

Licensed to AMPL Community Edition License for <cevikmelis14@gmail.com>.


In [16]:
from pyomo.environ import *

# create a model
model = ConcreteModel()

# define decision variables
model.x1 = Var(within=NonNegativeReals)
model.x2 = Var(within=NonNegativeReals)

# define objective function
model.profit = Objective(expr = 49000*model.x1 - model.x1**2 + 30*model.x2 - 2*model.x2**2 - 150*((model.x1/40)+(model.x2/20)), sense = maximize)

# define constraints
model.Constraint1 = Constraint(expr = model.x2 >= 0.5*model.x1)
model.Constraint2 = Constraint(expr = model.x1/40 + model.x2/20 <= (7100 * 8 * 5))

# Solve
solver = SolverFactory('knitro')
solver.solve(model)

print("Optimal Solution:")
print(f"X1: {model.x1.value}")
print(f"X2: {model.x2.value}")
print(f"Profit: {model.profit()}")

Optimal Solution:
X1: 16335.833334288951
X2: 8167.916667181691
Profit: 400289176.0404516


### **Edit and Run the code for the mathematical model produced by GPT4 (Circumstantial)**